# Spatio-temporal analysis of the actin fusion focus component in mutants 
### Determination of th signals center distances via gaussian fitting
For each channel, the distance between the fluorescent profile center and the origin is determined before computing RB, RG and BG distances over time for each event. Plotting is included. 

Used after Fiji data extraction (see README.md)

#### Folders architecture: 
Data for each POI in a given background/condition is stored in a folder contaning Images subfolder (corresponding to each timelapses from which data was extracted), each event information is recorded in event_id subfolders containing all channels focus fluorescent profiles for each timepoint and cytosolic fluorescence profile overtime. 


#### Package required:
-trajalign from Dr. A.Picco (https://apicco.github.io/trajectory_alignment/)

-numpy

-pandas

-matplotlib

-lmfit

#### Inputs required:
-path: path to the folder containing the mutants-POI subfolders

-foldername: list containing the names of the folders that need to be processed,
each should contain a POI in a given condition (e.g: 'POI-mutant1')

-colPOI: list of colors associated with each conditions for plotting

In [ ]:
path='/pathtoanalysisfolder' #folder containing the POI subfolders on which to run the alignment
foldername = ['Folder1','Folder2']  # list of POI subfolders to treat
colPOI=['blue','red']

In [ ]:
#import python packages
from trajalign.traj import Traj
from trajalign.average import load_directory
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import lmfit
from lmfit.models import GaussianModel
from lmfit.models import VoigtModel
from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
def replicats_nb(foldername):
    rep_list = [ folder for folder in os.listdir(os.path.join(path,foldername)) if 'Rep_' in folder ]
    return rep_list

def get_data(path):
    directory_rep = os.listdir(path)
    f = [ folder for folder in directory_rep if 'Image-' in folder ]
    event=[[] for _ in f]
    for i in range(len(f)):
        dir_rep = os.listdir(os.path.join(path,f[i]))
        event[i]=[ folder for folder in dir_rep if 'event_' in folder ]
    return f, event

img=[[] for _ in range(len(foldername))]
events=[[] for _ in range(len(foldername))]
Rep= []

for j in range(len(foldername)):
    rep=replicats_nb(foldername[j])
    Rep.append(rep)
    
for i in range(len(Rep)):
    img[i]=[[] for _ in range(len(Rep[i]))] 
    events[i]=[[] for _ in range(len(Rep[i]))]
    
for i in range(len(foldername)):  
    for j in range(len(Rep[i])):
        img[i][j], events[i][j]=get_data(os.path.join(path,foldername[i],Rep[i][j]))  

In [ ]:
#determine fusion frame based on cytoplasmic fluorescence intensity
def fusion_frame (path):
    m_c = load_directory(path,pattern='cyto',  comment_char = '#' ,sep= ',', frames = 0 , f = 1 )
    for x in m_c:
        n=len(x)   # number of frames
        U=np.empty( ((n-2),2) ) # creation of an empty array of n-2 lines and 2 columns
        
            
        for tau in range ((n-2)):  
# construction of the model for frame = x            
            m1 = np.mean(x.f()[0:tau])   
            m2 = np.mean(x.f()[(tau+1):n])
            m = np.array([m1] * tau + [m2]* (n-tau)) # use np.array because python can't substract list
            e = x.f() - m                           # substract theorical model from data
            U[tau,0] = x.frames()[tau]               # first column of matrix = frame number
            U[tau,1] = np.sum(e**2)                  # second column of matrix = sum of square errors 
        pos= np.argmin( U[:,1])              # get position of frame corresponding to mimimum sum of squared errors
        f_f=x.frames()[pos]              #get frame of fusion
        f_f= f_f-1   #correct for diff frame numeration PT/Fiji
        
    return f_f 

In [ ]:
#parameters obtained: fluorescence = heigth gaussian, mu= central value/centroid signal,FWHM= dispersion along line, 
#AUC= area under curve estimated from amplitude 

def gaussian_fit(path):
    #load fluoresence profile
    df=pd.read_csv(path)
    df.columns=("frame","fluorescence")
    if '_B' in path:
        col='blue'
    elif '_R' in path:
        col= 'red'
    else:
        col='lime'
        
    #fit gaussian distribution
    y=df.fluorescence
    x=df.frame
    model = GaussianModel()
    pars = model.guess(y,x)
    result = model.fit(y, x=x, params=pars)
    best_fit=result.best_fit
    
    
    if result.rsquared>0.6:
        center=result.params['center'].value/15.4321*1000 #get center gaussian fit
        f=result.params['height'].value
        
        Amp=result.params['amplitude'].value  #get amplitude gaussian fit
        s=result.params['sigma'].value #get sigma gaussian fit
        area=Amp*s*np.sqrt(2*np.pi)           #AUC
        
        fwhm=result.params['fwhm'].value #get full width at half max (FWHM) gaussian fit
        
    else:
        center= np.nan 
        f=np.nan
        area=np.nan
        fwhm=np.nan
       
    return x,y,best_fit,f,center,area,fwhm
       

def estimate_dist(path):
    #get .csv file paths
    print(path)
    directory_rep=os.listdir(path)
    profiles= [ file for file in directory_rep if 'profile' in file ]
    pr_B=[ file for file in profiles if '_B' in file ]
    pr_R=[ file for file in profiles if '_R' in file ]
    pr_G=[ file for file in profiles if '_G.' in file ]
    
    
    #initiate list for final df containing frame, maximum measured fluorescence and estimated signal center distance from origin ROI line
    frame=[]
    max_f_B=[]
    max_f_R=[]
    max_f_G=[]
    est_dist_B=[]
    est_dist_R=[]
    est_dist_G=[]
    width_B=[]
    width_R=[]
    width_G=[]
    AUC_B=[]
    AUC_R=[]
    AUC_G=[]
    
    #get max fluorescence and estimated distance to ROI line originby gaussian fit as well as fusion time
    ff=fusion_frame(path)
    fig=plt.figure( figsize = ( 25 , 5 ) )
    ct=1
    for i in range (len(pr_B)): 
        x_B,y_B,fit_B,f_B,center_B,auc_b,fwhm_b=gaussian_fit(os.path.join(path,pr_B[i]))
        x_R,y_R,fit_R,f_R,center_R,auc_r,fwhm_r=gaussian_fit(os.path.join(path,pr_R[i]))
        x_G,y_G,fit_G,f_G,center_G,auc_g,fwhm_g=gaussian_fit(os.path.join(path,pr_G[i]))
        if  i==(ff-10) or i==(ff-3)or i==(ff+1):
            plt.subplot(1,3,ct)
            plt.plot(x_B,y_B,color='cyan',alpha=0.5)
            plt.plot(x_R,y_R,color='indianred',alpha=0.5)
            plt.plot(x_G,y_G,color='lime',alpha=0.5)
            plt.plot(x_B,fit_B,color='black',linestyle='dashed')
            plt.plot(x_R,fit_R,color='maroon',linestyle='dashed')
            plt.plot(x_G,fit_G,color='darkgreen',linestyle='dashed')
            plt.xlabel('distance from origin (pxl)')
            plt.ylabel('fluorescence(AU)')
            ct+=1
          
        frame.append(i)
        max_f_B.append(f_B)
        est_dist_B.append(center_B)
        width_B.append(fwhm_b)
        AUC_B.append(auc_b)
        
        max_f_R.append(f_R)
        est_dist_R.append(center_R)
        width_R.append(fwhm_r)
        AUC_R.append(auc_r)
        
        max_f_G.append(f_G)
        est_dist_G.append(center_G)
        width_G.append(fwhm_g)
        AUC_G.append(auc_g)
    
    # create dataframe
    

    data=pd.DataFrame({'frame':frame,
                    'max_fluorescence_B': max_f_B,
                    'est_distance_B':est_dist_B,
                    'width_B':width_B,
                    'AUC_B':AUC_B,
                    'max_fluorescence_R': max_f_R,
                    'est_distance_R':est_dist_R,
                    'width_R':width_R,
                    'AUC_R':AUC_R,
                    'max_fluorescence_G': max_f_G,
                    'est_distance_G':est_dist_G,
                    'width_G':width_G,
                    'AUC_G':AUC_G})
    
     #compute directed distances 
    if (data.est_distance_B.mean()<data.est_distance_R.mean()):
        data['dRB']=data.est_distance_R-data.est_distance_B
        data['dRG']=data.est_distance_R-data.est_distance_G
        data['dBG']=data.est_distance_G-data.est_distance_B
    else:
        data['dRB']=data.est_distance_B-data.est_distance_R
        data['dRG']=data.est_distance_G-data.est_distance_R
        data['dBG']=data.est_distance_B-data.est_distance_G
        
    data.max_fluorescence_B=(data.max_fluorescence_B-min(data.max_fluorescence_B))/(max(data.max_fluorescence_B)-min(data.max_fluorescence_B))
    data.max_fluorescence_R=(data.max_fluorescence_R-min(data.max_fluorescence_R))/(max(data.max_fluorescence_R)-min(data.max_fluorescence_R))
    data.max_fluorescence_G=(data.max_fluorescence_G-min(data.max_fluorescence_G))/(max(data.max_fluorescence_G)-min(data.max_fluorescence_G))
    
    #align in time
    data.frame=data.frame-ff
    
    return data,fig

def write_pdf(fname, figures):
    doc = PdfPages(fname)
    figures.savefig(doc, format='pdf')
    doc.close()

In [ ]:
df=[[] for _ in range(len(img))]
for i in range(len(img)):
    for j in range(len(img[i])):
        for k in range(len(img[i][j])):
            for ev in events[i][j][k]:
                path_ev=os.path.join(path,foldername[i],Rep[i][j],img[i][j][k],ev)
                data,fig=estimate_dist(path_ev)
                data.to_csv(os.path.join(path_ev,'profiles_dist_fluo.csv'),index=False)
                df[i].append(data)
                write_pdf(os.path.join(path_ev,'profiles_fluo_GaussianModel.pdf'),fig)

In [ ]:
def mad (s):
    list_s=[]
    m=np.nanmedian(s)
    for x in s: 
        s_i=np.absolute(x-m)
        list_s.append(s_i)
   
    mad_s= np.nanmedian(list_s)
    mad=mad_s*1.4826
    return mad

def mad_ln( s ) :
    sigma_ln = mad( np.log( s ) )
    m = np.nanmedian( s )
    sigma = m * sigma_ln
    return sigma

# function for distance and max fluorescence average

def average (df_list,filename):
    f_min =0 
    f_max= 0
    for df in df_list: 
        if  df.frame.min()< f_min :
            f_min = df.frame.min()
        if df.frame.max()> f_max :
            f_max = df.frame.max()
    
    frames=[]
    med_dRB=[]
    med_dRG=[]
    med_dBG=[]
    med_FluoR=[]
    med_FluoG=[]
    med_FluoB=[]
    med_width_B=[]
    med_width_R=[]
    med_width_G=[]
    
    
    mad_dRB=[]
    mad_dRG=[]
    mad_dBG=[]
    mad_FluoR=[]
    mad_FluoG=[]
    mad_FluoB=[]
    mad_width_B=[]
    mad_width_R=[]
    mad_width_G=[]
    
    for frame in range(f_min,f_max):
        frames.append(frame)
        dRB=[]
        dRG=[]
        dBG=[]
        FluoR=[]
        FluoG=[]
        FluoB=[]
        WB=[]
        WR=[]
        WG=[]
        for df in df_list:
            for i in range(len(df.frame)):
                if df.frame.iloc[i]==frame:
                    dRB.append(df.dRB.iloc[i])
                    dRG.append(df.dRG.iloc[i])
                    dBG.append(df.dBG.iloc[i])
                    FluoR.append(df.max_fluorescence_R.iloc[i])
                    FluoG.append(df.max_fluorescence_G.iloc[i])
                    FluoB.append(df.max_fluorescence_B.iloc[i])
                    WB.append(df.width_B.iloc[i])
                    WR.append(df.width_R.iloc[i])
                    WG.append(df.width_G.iloc[i])
               
        med_dRB.append(np.nanmedian(dRB))
        med_dRG.append(np.nanmedian(dRG))
        med_dBG.append(np.nanmedian(dBG))
        med_FluoR.append(np.nanmedian(FluoR))
        med_FluoG.append(np.nanmedian(FluoG))
        med_FluoB.append(np.nanmedian(FluoB))
        med_width_B.append(np.nanmedian(WB))
        med_width_R.append(np.nanmedian(WR))
        med_width_G.append(np.nanmedian(WG))
        
        mad_dRB.append(mad(dRB))
        mad_dRG.append(mad(dRG))
        mad_dBG.append(mad(dBG))
        mad_FluoR.append(mad_ln(FluoR))
        mad_FluoG.append(mad_ln(FluoG))
        mad_FluoB.append(mad_ln(FluoB))
        mad_width_B.append(mad_ln(WB))
        mad_width_R.append(mad_ln(WR))
        mad_width_G.append(mad_ln(WG))
        
    time=[]
    for f in frames:
        t=3*f
        time.append(t)
    data=pd.DataFrame({'frame':frames,
                       'time':time,
                    'med_fluoB': med_FluoB,
                    'mad_fluoB':mad_FluoB,
                    'med_fluoR': med_FluoR,
                    'mad_fluoR':mad_FluoR,  
                    'med_fluoG': med_FluoG,
                    'mad_fluoG':mad_FluoG,
                    'med_dRB':med_dRB,
                    'mad_dRB':mad_dRB,
                    'med_dRG':med_dRG,
                    'mad_dRG':mad_dRG,
                    'med_dBG':med_dBG,
                    'mad_dBG':mad_dBG,
                    'med_width_B':med_width_B,
                    'mad_width_B':mad_width_B,
                      'med_width_R':med_width_R,
                    'mad_width_R':mad_width_R,
                      'med_width_G':med_width_G,
                    'mad_width_G':mad_width_G})
    data.to_csv(filename, index=False)                
    return data  

In [ ]:
df_med=[[] for _ in range(len(df))]
for n in range(len(df)):
    path_img=os.path.join(path,foldername[n],'Medial_data')
    filename=os.path.join(path_img, 'med_'+foldername[n]+'.csv')
    os.makedirs(path_img, exist_ok=True)
    df_med[n]=average(df[n],filename) 

In [ ]:
def plot_dist (df_list,colPOI,names):
    plt.grid(visible=True)
    for i in range(len(df_list)):
        df=df_list[i]
        plt.plot(df.time,df.med_dRG,color=colPOI[i],label=names[i],linewidth=9)
        plt.fill_between(df.time,df.med_dRG-df.mad_dRG,df.med_dRG+df.mad_dRG,color=colPOI[i], alpha =0.2 )
        plt.plot(df.time,df.med_dRG-df.mad_dRG,color=colPOI[i],linestyle='--',alpha=0.3)
        plt.plot(df.time,df.med_dRG+df.mad_dRG,color=colPOI[i],linestyle='--',alpha=0.3)
    plt.ylim(-100,150)
    plt.ylabel ('median distance from Myo_P (nm)',fontsize=28)
    plt.hlines(0,-60,5, color='red',linewidth=4)
    handles, labels = plt.gca().get_legend_handles_labels()
    labels, ids = np.unique(labels, return_index=True) 
    handles = [handles[i] for i in ids] 
    plt.legend(handles, labels, loc='best',fontsize=28) 

    plt.xlim(-60,5)
    plt.xlabel ('time from fusion (min)',fontsize=28)
    plt.xticks(fontsize=26)
    plt.yticks(fontsize=26)

In [ ]:
plt.figure(figsize=((30,10)))
plot_dist(df_med,colPOI,foldername)
figname=os.path.join(path_med,'distances.pdf')
plt.savefig(figname)
plt.show()